# Integrate AMPIX .h5 files created with the DanMAX AMPIX battery script using pyFAI

This notebook can be used to integrate a series of `*.h5` files with `pyFAI` from the master file. The frames in a given entry are averaged before the integration. The script must be run individually for each cell in the whole measurement run. The required inputs are:

1 `fname`: Path for `.h5` master file<br>
2 `poni`: Path for `.poni`<br>
3 `maskPath`: Path for detector mask, e.g. `.npy` format<br>
4 `nbins`: Number of bins in the integrated data<br>
5 `useQ`: `True`/`False` if true the integration will be done in Q [Å^{-1}] - if false the integration is in 2theta [degrees]<br>
6 `polarization_factor`: Polarization factor, should be very close to (but slightly lower than) 1. <br>

The output will be saved in:
`/data/visitors/danmax/PROPOSAL/VISIT/process/pyFAI/SAMPLE/scan-XXXX_pilatus_integrated.h5` i.e. mirroring the location of the raw data. A timestamp file with the real times is also generated and saved in the h5-file and as a
`*.text` file. The times correspond approximately to halfway through a timescan on the cell in order to have a single timestamp per timescan as this is the common practice.


In [ ]:
%matplotlib widget
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import pyFAI
import glob
from datetime import datetime
import pytz
import re

fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/battery_X/master.h5'
poni = '/data/visitors/danmax/PROPOSAL/VISIT/process/MyPoni.poni'
maskPath = '/data/visitors/danmax/PROPOSAL/VISIT/process/MyMask.npy'

nbins = 3000
polarization_factor=0.99997

useQ = True # Use Q/AA^-1 if True. Will use TTH in degrees if False

#--------------------------------------------------------

#Loading mask and poni to pyFAI
ai = pyFAI.load(poni) 
mask = np.load(maskPath) 
ai.set_mask(mask)

timezone = pytz.timezone('Europe/Stockholm') #The timezone is needed to generate the proper time stamp

#Setting up the filename for the integrated data
out = fname.split('.')[0]+'_ampix_pilatus_integrated.h5'
out = out.replace('raw', 'process/pyFAI')
path, _ = os.path.split(out)
if not os.path.exists(path):
    os.makedirs(path)
print('Integrated data will be saved in: {0}'.format(out))
new_filename = out.split('.')[0]+'_timestamps.txt'
stamplist = []

#Generating the file for the integrated data
out_fh = h5py.File(out, 'w')

#Reading the raw file
fh = h5py.File(fname)

#Here, we determine how many timescans have been made on the cell based on the number of generated Pilatus files
d = fname.split('master')[0]
nentry = len(glob.glob(d+'*.h5'))-1

#The data structure is set up based on the number of bins chosen and the number of timescans on the cell
dset = out_fh.create_dataset('I', shape=(nentry, nbins))

if useQ:
    print('Integrating in Q[AA-1]')
else:
    print('Integrating in TTH[Deg]')

for ent in range(nentry):
    images = fh['/entry'+format(ent+1, '04d')+'/instrument/pilatus/data']
    avg_img = np.mean(images, axis=0) #The frames in the given timescan are averaged.
    
    if useQ:
        q, I = ai.integrate1d(avg_img, nbins, unit='q_nm^-1', polarization_factor=polarization_factor, correctSolidAngle=True)
        dset[ent] = I
        if ent == 0:
            out_fh.create_dataset('q', data=q/10) # convert from nm^-1 to AA^-1
    else:
        tth, I = ai.integrate1d(avg_img, nbins, unit='2th_deg', polarization_factor=polarization_factor, correctSolidAngle=True)
        dset[ent] = I
        if ent == 0:
            out_fh.create_dataset('tth', data=tth)
    
    timeStampUnix = fh['/entry'+format(ent+1, '04d')+'/measurement/pcap_trigts'][round(len(timeStampsUnix)/2)] #The length of the timescan is determind and the frame halfway through is selected for the timestamp
    offsetTZ = timezone.localize(datetime.utcfromtimestamp(timeStampUnix)).utcoffset()
    stamp = "{0:05d}, {1}\n".format(ent+1, datetime.utcfromtimestamp(timeStampUnix)+offsetTZ) #The timescan number is saved along with the corresponding date and time
    stamplist.append(stamp)
    
    print('Processing: {0} of {1} - {2:.1f}% complete...'.format(ent+1, nentry, ent/nentry*100), end='\r')
    
np.savetxt(new_filename, stamplist, fmt="%s") #The time stamp list is generated and saved in the same folder as the h5 file containing the integrated data
    
with open(poni, 'r') as poni_file:
    print('\nWriting poni information to file...')
    p = out_fh.create_dataset('poni_file', data=poni_file.read()) #The used poni file is saved to the h5 file for future reference

print('Writing mask path to file...')
m = out_fh.create_dataset('mask', data=mask) #The used mask file is saved to the h5 file for future reference

print('Writing time stamp to file...')
t = out_fh.create_dataset('timestamp', data=stamplist) #The time stamp list is saved to the h5 file
    
out_fh.close()
print('All done!')